In [1]:
import boto3

In [2]:
sm=boto3.client('sagemaker')

## Batched pre-processing of raw data

In [111]:
tfj=sm.describe_transform_job(TransformJobName='200413-bac-dpp1-rpb-1-ccde820d5f7b48438a9b53c6dab88e670e9da5e9f')

In [19]:
temp=sm.create_transform_job(
    #jobname needs to be unique, append date/something
TransformJobName= 'boto3-dpp1-transformer-only', 
ModelName= tfj['ModelName'],
TransformInput=tfj['TransformInput'],
TransformOutput=tfj['TransformOutput'], #change output path
TransformResources= {'InstanceType': 'ml.m5.xlarge',
                    'InstanceCount':1}    
)

## Evaluate new model on held out dataset

### 1) Copy it into S3 and preprocess/transform

In [ ]:
!aws s3 cp bace-validation.csv s3://location_input_here

In [131]:
#create batch transform job for the holdout dataset
tfj['TransformInput']['DataSource']['S3DataSource']['S3Uri']=' s3://location_input_here'
tfj['TransformOutput']['S3OutputPath']=' s3://location_input_here/output'

In [133]:
sm= boto3.client('sagemaker')

temp=sm.create_transform_job(
    #jobname needs to be unique, append date/something
TransformJobName= 'boto3-dpp1-validation-holdout', 
ModelName= tfj['ModelName'],
TransformInput=tfj['TransformInput'],
TransformOutput=tfj['TransformOutput'], 
TransformResources= {'InstanceType': 'ml.m5.xlarge',
                    'InstanceCount':1}    
)

## 2) Generate predictions on new model using Batch transform job

In [5]:
from sagemaker.model import Model
#create model from training data
training_job_name='lambda-retrain2020-04-17-22-05-06'
newtj=sm.describe_training_job(TrainingJobName=training_job_name)
#use training job to create model
new_model=Model(
    name=newtj['TrainingJobName'],
    model_data=newtj['ModelArtifacts']['S3ModelArtifacts'],
    image=newtj['AlgorithmSpecification']['TrainingImage'],
    role= newtj['RoleArn'])

In [206]:
#batch transform job
from sagemaker.transformer import Transformer

trans=new_model.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    strategy='SingleRecord',
    output_path='s3://jw-spark/bace-autopilot/200413-bace-automl/validation-holdout/prediction_sdk')

trans.transform('s3://jw-spark/bace-autopilot/200413-bace-automl/validation-holdout-transformed/' ,content_type='application/x-recordio-protobuf')



In [153]:
import pandas as pd

In [ ]:
#download data using either aws s3 cp / or using the pd.read_csv 

In [158]:
#predictions from new model
pred=pd.read_csv('bace-validation-holdout.csv.out.out', header= None)

In [159]:
pred.head()

,0
0,6.583392
1,6.467030
2,5.872973
3,6.623354
4,7.346873


In [163]:
#actual target values
valid= pd.read_csv('bace-validation.csv', usecols=['pIC50'])
valid.head()

,pIC50
0,6.721246
1,10.522879
2,9.522879
3,9.522879
4,9.154901


In [167]:
from sklearn.metrics import mean_squared_error as mse
import numpy as np

In [169]:
#calculating RMSE
np.sqrt(mse(valid,pred))

1.6537821539174093